# NHL Fantasy Pool Scores

### Steps
<ul>
    <li> <b>Set Up:</b> Loads required packages, fantasy team draft and creates necessary empty lists and sets to run script</li>
    <li> <b>Step 1:</b> Pulls list of relevant games within requested timeframe</li>
    <li> <b>Step 2:</b> Stores a list of NHL Game Nights</li>
    <li> <b>Step 3:</b> Stores APIs Game IDS required to pull necessary stats</li>
    <li> <b>Step 4:</b> Sausage Factory; Gathers required player statistics to calculate Fantasy Pool standings</li>
    <li> <b>Step 5:</b> Output; Generates necessary dataframes and CSV files to produce final standings table</li>
</ul>

### Future Improvements
<ul>
    <li> Be able to auto-detect minor spelling diffferences between fantasy draft and API database to make JOINs</li>
</ul>

## Set Up

In [1]:
# Installation Guide for Sportsreference Python Package
## https://sportsreference.readthedocs.io/en/stable/installation.html

# Documentation for Sportsreference Python Package
## https://sportsreference.readthedocs.io/en/stable/nhl.html#

# Importing packages
from datetime import datetime,timedelta,time
from sportsreference.nhl.boxscore import Boxscores,Boxscore
import pandas as pd

# Loading in Fantasy Team Roster
fantasy_team = pd.read_csv("Fantasy_Round2.csv", encoding='utf-8')
fantasy_team['Player'] = fantasy_team['Player'].str.lower()

# Creating empty lists for data to be populated
game_days = [] # List of days with NHL games
game_ids = [] # List of Game IDs needed to pull game statistics
invalid_ids = set() # List of Game IDs that are not valid
data = [] # List of collected statistics that will be used to turn into a Dataframe

print("Set Up Completed: Pre-requisites are ready!")

Set Up Completed: Pre-requisites are ready!


## Steps 1 - 3

In [2]:
# Pulling in list of relevant games
games_list = Boxscores(datetime(2020, 8, 11),datetime.today())
print("Step 1 Completed: Pulled in list of relevant games")

# Storing in NHL Game Night dates into 'game_days' list
for date in games_list.games:
    game_days.append(date)
print("Step 2 Completed: Stored in NHL Game Night dates into 'game_days' list")

# Storing Game IDs, Home Team Name, Away Team Name and date as a list into 'game_ids' list 
dupes_check = []
for date in game_days:
    for gameid in games_list.games[date]:
        gm_list = []
        if gameid['winning_name'] is None:
            invalid_ids.add(gameid['boxscore'])
        else:
            dupes_check.append(gameid['boxscore'])
            if dupes_check.count(gameid['boxscore'])<= 1:
                gm_list.append(gameid['boxscore'])
                gm_list.append(gameid['home_abbr'])
                gm_list.append(gameid['away_abbr'])
                gm_list.append(date)
                game_ids.append(gm_list)
            else:
                pass
print("Step 3 Completed: Stored Game IDs into 'game_ids' list")

# Cleaning up 'game_ids' list to remove duplicates        
game_ids = list(set(map(tuple,game_ids )))

# Cleaning up 'game_ids' list to remove blank ids
for ids in game_ids:
    if len(ids[0]) == 0:
        game_ids.remove(ids)

print("Step 4 can be started: Clean up completed")

Step 1 Completed: Pulled in list of relevant games
Step 2 Completed: Stored in NHL Game Night dates into 'game_days' list
Step 3 Completed: Stored Game IDs into 'game_ids' list
Step 4 can be started: Clean up completed


## Step 4

In [3]:
# Sausage Factory

## Creating a loop that pulls required player statistics for each game_id
for game in game_ids:
    # Home Players
    ## Storing a list of elements containing name, hometeam, game_id, date, goals, assists, points into 'data' list
    try:
        home_players = Boxscore(game[0]).home_players
    except Exception:
        invalid_ids.add(game[0]) #use a set so you don't keep any duplicates.
        pass
    else:
        for hplayer in home_players:
            hm_stat = []
            hm_stat.append(hplayer.name)
            hm_stat.append(game[1]) 
            hm_stat.append(game[0]) #gameid
            hm_stat.append(game[3])
            hm_stat.append(hplayer.goals)
            hm_stat.append(hplayer.assists)
            hm_stat.append(hplayer.points)
            data.append(hm_stat)
    
    # Away Players
    ## Storing a list of elements containing name, awayteam, game_id, date, goals, assists, points into 'data' list
    try:
        away_players = Boxscore(game[0]).away_players
    except Exception:
        invalid_ids.add(game[0])
        pass
    else:
        for aplayer in away_players:
            aw_stat = []
            aw_stat.append(aplayer.name)
            aw_stat.append(game[2])
            aw_stat.append(game[0])
            aw_stat.append(game[3])
            aw_stat.append(aplayer.goals)
            aw_stat.append(aplayer.assists)
            aw_stat.append(aplayer.points)
            data.append(aw_stat)
print("Step 4 Completed: Pulled player statistics into 'data' list")


Step 4 Completed: Pulled player statistics into 'data' list 
Step 5 Started: Creating CSV files for Fantasy Pool Standing


## Step 5A

In [4]:
# Transforming the 'data' list into a dataframe and saving it as a CSV
hockey_stats_ts = pd.DataFrame (data, columns = ['Player','Team','Game_ID','Date','Goals','Assists','Points'])
hockey_stats_ts['Date'] = hockey_stats_ts['Date'].apply(pd.to_datetime)

### Converting Names to Match Fantasy Draft Picks
hockey_stats_ts['Player_Last'] = hockey_stats_ts['Player'].str.split(" ").str[1]
hockey_stats_ts['Player_First'] = hockey_stats_ts['Player'].str[:1]
hockey_stats_ts['Player'] = hockey_stats_ts['Player_Last'].str.cat(hockey_stats_ts['Player_First'], sep = " ").str.lower()

hockey_stats_ts.to_csv('nhl_player_statistics.csv')
print('\nDetailed NHL player stats have been generated as nhl_player_statistics.csv')

# Grouping up the details stats table above into cumulative player stats and saving it as a CSV
hockey_stats_cum = pd.DataFrame(hockey_stats_ts.groupby(['Player','Team']).sum()).reset_index()
hockey_stats_cum.to_csv('CUM_nhl_player_statistics.csv')
print('Cumulative NHL player stats have been generated as CUM_nhl_player_statistics.csv')

# Creating the final dataset that will aggregate points and assign it to the Fantasy Team with ranking

## Creating two dataframe slices summing results up until today()-1 and today()-2
recent_raw = hockey_stats_ts.loc[hockey_stats_ts['Date'] <= (datetime.today()- timedelta(days=1)).replace(hour = 0, minute = 0, second = 0)]
previous_raw = hockey_stats_ts.loc[hockey_stats_ts['Date'] <= (datetime.today()- timedelta(days=2)).replace(hour = 0, minute = 0, second = 0)]


## Joining the recent_raw slice to previous_raw slice and renaming columns
results = pd.merge(pd.DataFrame(recent_raw.groupby(['Player']).sum()).reset_index(), pd.DataFrame(previous_raw.groupby(['Player']).sum()).reset_index(), how='left',left_on='Player', right_on='Player')
results.columns = ['Player','Recent_Goals','Recent_Assists','Recent_Points','Previous_Goals','Previous_Assists','Previous_Points']


## Joining the fantasy team roster to results dataframe
results_standing = pd.merge(fantasy_team, results, how='left', left_on='Player', right_on='Player')

# Looking for player names that are not matched
print('Checking For Missing Players between Roster and Player Statistics...')
if pd.DataFrame(results_standing['Player'].loc[results_standing['Previous_Points'].isna()]).empty:
    print('Players matched...Continue to next step')
else:
    print('Players missing - Please confirm these are only players who did not play... Validate before proceeding to next step')
    print(pd.DataFrame(results_standing['Player'].loc[results_standing['Previous_Points'].isna()]))




Detailed NHL player stats have been generated as nhl_player_statistics.csv
Cumulative NHL player stats have been generated as CUM_nhl_player_statistics.csv
Checking For Missing Players between Roster and Player Statistics...
Players missing - Please confirm these are only players who did not play... Validate before proceeding to next step
             Player
93        toffoli t
162       stamkos s
163  van riemsdyk j


## Step 5B

In [5]:
## Aggregating measures by Fantasy Team
results_cum = pd.DataFrame(results_standing.groupby(['Fantasy']).sum()).reset_index()

## Creating columns to calculate Fantasy Team recent rank, previous rank, change in rank, change in points
results_cum['Current_Rank'] = results_cum['Recent_Points'].rank(method='max', ascending = False)
results_cum['Previous_Rank'] = results_cum['Previous_Points'].rank(method='max', ascending = False)
results_cum['Rank_Change'] = results_cum['Current_Rank'] - results_cum['Previous_Rank'] 
results_cum['Point_Change'] = results_cum['Recent_Points'] - results_cum['Previous_Points']

## Creating a truncated dataframe ordered by recent rank
standings = results_cum[['Current_Rank','Fantasy','Recent_Points','Rank_Change','Point_Change']].sort_values(by=['Current_Rank'], ascending=True)
standings.columns = ['Rank','Fantasy Team','Points','Rank Change','Points Gained']
standings.to_csv('Fantasy_Pool_Standing.csv')
print('\nResults of your fantasy hockey pool has been generated in fantasy_nhl_pool_standing.csv \nStep 5 Completed: Updated Hockey Pool Standing.. Process Completed')


Results of your fantasy hockey pool has been generated in fantasy_nhl_pool_standing.csv 
Step 5 Completed: Updated Hockey Pool Standing.. Process Completed
